In [2]:
import pandas as pd

In [5]:
data = pd.read_csv('C:\\Users\\wz\Downloads\\AdSmartABdata - AdSmartABdata.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8077 entries, 0 to 8076
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   auction_id   8077 non-null   object
 1   experiment   8077 non-null   object
 2   date         8077 non-null   object
 3   hour         8077 non-null   int64 
 4   device_make  8077 non-null   object
 5   platform_os  8077 non-null   int64 
 6   browser      8077 non-null   object
 7   yes          8077 non-null   int64 
 8   no           8077 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 568.0+ KB


In [7]:
for column in data.drop(['auction_id','hour','device_make'],axis=1).columns:
    print(column,'-',data[column].unique())

experiment - ['exposed' 'control']
date - ['2020-07-10' '2020-07-07' '2020-07-05' '2020-07-03' '2020-07-09'
 '2020-07-04' '2020-07-06' '2020-07-08']
platform_os - [6 5 7]
browser - ['Chrome Mobile' 'Chrome Mobile WebView' 'Facebook' 'Mobile Safari'
 'Chrome Mobile iOS' 'Samsung Internet' 'Mobile Safari UI/WKWebView'
 'Chrome' 'Opera Mini' 'Edge Mobile' 'Android' 'Pinterest' 'Opera Mobile'
 'Firefox Mobile' 'Puffin']
yes - [0 1]
no - [0 1]


In [8]:
data['experiment'].value_counts()#试验组与控制组样本数量

control    4071
exposed    4006
Name: experiment, dtype: int64

In [34]:
from scipy.stats import norm
import math as mt

In [35]:
def cal_pvalue(z_score):
    return scipy.stats.norm.sf(abs(z_score))#survival functiion:1-cdf=上侧分位数
def get_z_score(alpha):
    return norm.ppf(alpha)#percent point function根据cdf求解分位数
def cal_propotion_confidence(p1,p2,p,n1,n2,alpha):
    d = round(p2-p1,4)
    sd = mt.sqrt(p*(1-p)*(1/n1+1/n2))
    z_score = d/sd
    p_value = round(cal_pvalue(z_score)*2,6)
    ME = round(get_z_score(1-alpha/2)*sd,4)
    print("P-value 为",p_value,"置信区间为[",d-ME,",",d+ME,"]","试验产生的变化值",d*100,"%")

In [36]:
control = data[data['experiment']=='control']
exposed = data[data['experiment']=='exposed']
total_control = control['auction_id'].nunique()#去重id数
total_exposed = exposed['auction_id'].nunique()

clicks_control = control['auction_id'].loc[(control['yes']==1) | (control['no']==1)].nunique()
clicks_exposed = exposed['auction_id'].loc[(exposed['yes']==1) | (exposed['no']==1)].nunique()

p1 = clicks_control/total_control
p2 = clicks_exposed/total_exposed
p = (clicks_control+clicks_exposed)/(total_control+total_exposed)

cal_propotion_confidence(p1,p2,p,total_control,total_exposed,0.05)

P-value 为 0.012316 置信区间为[ 0.004400000000000001 , 0.0358 ] 试验产生的变化值 2.01 %
